# Read Before Using This File

This code requires the orginal physionet EEG data which can be foudn here: https://physionet.org/content/chbmit/1.0.0/.

This code also requires the **seizure_locations.csv** and **new_locations.csv** files to work correctly. 

# Feature Extraction

## Importing the Packages we Need 

In [ ]:
# Firstly lets import the packages we need 
!pip install mne 
# Note: the rest of the packages should be pre installed on colab 

     |████████████████████████████████| 6.6MB 3.2MB/s 


In [ ]:
# Now lets import the packages we need 
import os # Needed for navigating directories
import csv # Need to read the csv files 
import numpy as np
import mne 
import matplotlib.pyplot as plt

# Question about TF analysis website 
# https://mail.nmr.mgh.harvard.edu/pipermail//mne_analysis/2018-December/005398.html

## Main Feature Extracting Function 

In [ ]:
# This function will be used to create a images of 60 secs using a entire file or a given start and end time

def get_images_from_file(file_location, common_channels, fs=256, notch_filter_value=60, tmin=1, tmax=2, entire_file=True):
    # Note: requies the mne and numpy modules 
    # Input - takes file location, fs(sample rate of file)(default =256Hz), 
    # common_channels(required)- list of strings indicating the channels common to all eeg files
    # notch_filter_value (at what hertz to apply the notch filter)(default = 60Hz)
    # Ouput - will be (no_of_epcohs, 3(for channels),230(for freqs), (230 for times) )
    # i.e. a "image" with 3 channels that is 230 * 230 

    # Reads the raw data and saves it into memory
    raw = mne.io.read_raw_edf(file_location, preload=True)

    if entire_file == False: 
        raw.crop(tmin=tmin, tmax=tmax, include_tmax=True)

    # Applys the notch filter at the given hertz range 
    raw.notch_filter(freqs=notch_filter_value)

    # Segments the raw by 1 mine (60 secs) and loads into memory 
    epochs = mne.make_fixed_length_epochs(raw, duration=60, preload=True)
    

    # Loads the epochs into memory so we can use the reorder opertation in the next line of code
    epochs.load_data()

    # Only keeps the common channels and reorders the channels to  matches the order of ch_names
    epochs.pick_channels(ch_names=common_channels, ordered=True)

    # Sets up the frequency array to pass into the next line
    freqs = np.arange(5,120,.5)

    # Info on the the tfr_morlet can be found here: https://mne.tools/stable/generated/mne.time_frequency.tfr_morlet.html#mne.time_frequency.tfr_morlet
    # Lets explain some of the paramters freqs - is the freqs along the y axis to be calculated for 
    # decim - is is the time points along x to be calulated for default = 1 (this gives 230 in the last axis of the array
    # as 256(freq) * 60 sec ≈ 230)
    # use_fft - as we want to use the fast fouier transform for efficientcy 
    # average - is False as we DONT want to average the values across epochs 
    # return_itc - is False as we dont care about the inter-trial coherence (ITC) whatever that is
    # n_cycles - is 60 just beacuse it works (tbh not sure about this value)
    power = mne.time_frequency.tfr_morlet(epochs, freqs=freqs, n_cycles=60, decim=67, 
                                             use_fft=True, average=False, return_itc=False)
    
    # Change the baseline otherwise the values will not make much sense as the lower bands will over power 
    # not to sure why the exact values where used but I found it on the mne tutoiral found 
    # here: https://mne.tools/stable/auto_tutorials/time-freq/plot_sensors_time_frequency.html#sphx-glr-auto-tutorials-time-freq-plot-sensors-time-frequency-py
    # so I thought it made sense to use
    power.apply_baseline(baseline=(-0.5, 0), mode='logratio')

    return power

## Global Variables 

In [ ]:
# Global variables 

# We are using these common channles as in these 3 channels have high intra-
# patient activation during seizures (the 3 channels are FP2-F8, F8-T8, and T8-P8)
# may be  T8-P8-0 instead of T8-P8 in this dataset 
common_channels = ['FP2-F8', 'F8-T8', 'T8-P8-0']

EEG_data_file_path = "/content/drive/My Drive/Research/data/EEG/" # Place location of raw EEG data here (The data from the CHB-MIT datset)

seizure_location_file_path = "/content/drive/My Drive/Research/data/seizure_locations.csv" # Points to seizure_locations.csv path 

new_locations_file_path = "/content/drive/My Drive/Research/data/new_locations.csv" # Points to new_locations.csv 

## Getting files with seizure samples and long enough seizure samples 

Get all siezure file names

In [ ]:
# Just gets all the file names that contain sezirues and places into the seizure_file array 
seizure_files = [] # Contains list of strings with all files have seizure 
import csv
with open(seizure_location_file_path, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        seizure_files.append(row[0])
seizure_files

['chb01_03.edf',
 'chb01_04.edf',
 'chb01_15.edf',
 'chb01_16.edf',
 'chb01_18.edf',
 'chb01_21.edf',
 'chb01_26.edf',
 'chb02_16.edf',
 'chb02_16+.edf',
 'chb02_19.edf',
 'chb03_01.edf',
 'chb03_02.edf',
 'chb03_03.edf',
 'chb03_04.edf',
 'chb03_34.edf',
 'chb03_35.edf',
 'chb03_36.edf',
 'chb04_05.edf',
 'chb04_08.edf',
 'chb04_28.edf',
 'chb04_28.edf',
 'chb05_06.edf',
 'chb05_13.edf',
 'chb05_16.edf',
 'chb05_17.edf',
 'chb05_22.edf',
 'chb06_01.edf',
 'chb06_01.edf',
 'chb06_01.edf',
 'chb06_04.edf',
 'chb06_04.edf',
 'chb06_09.edf',
 'chb06_10.edf',
 'chb06_13.edf',
 'chb06_18.edf',
 'chb06_24.edf',
 'chb07_12.edf',
 'chb07_13.edf',
 'chb07_19.edf',
 'chb08_02.edf',
 'chb08_05.edf',
 'chb08_11.edf',
 'chb08_13.edf',
 'chb08_21.edf',
 'chb09_06.edf',
 'chb09_08.edf',
 'chb09_08.edf',
 'chb09_19.edf',
 'chb10_12.edf',
 'chb10_20.edf',
 'chb10_27.edf',
 'chb10_30.edf',
 'chb10_31.edf',
 'chb10_38.edf',
 'chb10_89.edf',
 'chb11_82.edf',
 'chb11_92.edf',
 'chb11_99.edf',
 'chb12_06.ed

Get all siezure samples with large enough seizures

In [ ]:
# Get all the large seizure segments file, start time and end time and places into array bellow 
large_seizure_files_with_time  = []
large_seizure_files_names = []
seizure_files = [] # Contains list of strings with all files have seizure 
with open(new_locations_file_path, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        large_seizure_files_with_time.append([row[0], int(row[1]), int(row[2])])
        large_seizure_files_names.append(row[0])
large_seizure_files_names


['chb01_18.edf',
 'chb01_21.edf',
 'chb01_26.edf',
 'chb02_16.edf',
 'chb03_02.edf',
 'chb03_03.edf',
 'chb03_35.edf',
 'chb04_08.edf',
 'chb04_28.edf',
 'chb04_28.edf',
 'chb05_06.edf',
 'chb05_13.edf',
 'chb05_16.edf',
 'chb05_17.edf',
 'chb05_22.edf',
 'chb07_12.edf',
 'chb07_13.edf',
 'chb07_19.edf',
 'chb08_02.edf',
 'chb08_05.edf',
 'chb08_11.edf',
 'chb08_13.edf',
 'chb08_21.edf',
 'chb09_06.edf',
 'chb09_08.edf',
 'chb09_08.edf',
 'chb09_19.edf',
 'chb10_20.edf',
 'chb10_27.edf',
 'chb10_31.edf',
 'chb10_38.edf',
 'chb11_99.edf',
 'chb12_06.edf',
 'chb12_23.edf',
 'chb12_23.edf',
 'chb13_21.edf',
 'chb13_40.edf',
 'chb13_59.edf',
 'chb13_62.edf',
 'chb13_62.edf',
 'chb15_06.edf',
 'chb15_15.edf',
 'chb15_22.edf',
 'chb15_28.edf',
 'chb15_31.edf',
 'chb15_40.edf',
 'chb15_40.edf',
 'chb15_40.edf',
 'chb15_46.edf',
 'chb15_49.edf',
 'chb15_52.edf',
 'chb15_54.edf',
 'chb15_54.edf',
 'chb15_54.edf',
 'chb15_62.edf',
 'chb17a_03.edf',
 'chb17a_04.edf',
 'chb17b_63.edf',
 'chb18_31.

## Feature extraction for seizure samples

In [ ]:
# Temporary the fetaure extracted images  
seizure_train_data = []
seizure_test_data = [] 
# Patient 15 onwards will be used for testing the other patients will be ued for training 

error_files = [] # The names of the files the caused errors will be written here 

# Gets all the patient dirs in the EEG dataset 
dirs = [dir for dir in os.listdir(EEG_data_file_path) if os.path.isdir(os.path.join(EEG_data_file_path,dir))]

for directory in dirs: # Loops over all the directories
    for filename in os.listdir(EEG_data_file_path + directory): # Loops over each file in directory
        if (filename.endswith(".edf")): # Sees if file is a edf (i.e does it contain eeg data)
            # Is file a file that contains large seizure 
            for f in large_seizure_files_with_time: 
                if f[0] == filename: 
                    if int(f[0][3:5]) >= 15: # Checks if patient number is 15 or greater 
                        try: 
                            print("The file being converted is", f[0])
                            power = get_images_from_file(EEG_data_file_path + directory + "/" + filename, common_channels, tmin=f[1], tmax=f[2], entire_file=False)
                            seizure_test_data.append(power) # Append value to seizure 
                        except Exception as e:
                            print(e)  
                            error_files.append(f[0]) # Append filename that causes error
                    else: 
                        try: 
                            print("The file being converted is", f[0])
                            power = get_images_from_file(EEG_data_file_path + directory + "/" + filename, common_channels, tmin=f[1], tmax=f[2], entire_file=False)
                            seizure_train_data.append(power) # Append value to seizure 
                        except Exception as e:
                            print(e)  
                            error_files.append(f[0]) # Append filename that causes error
                    

The file being converted is chb01_26.edf
Extracting EDF parameters from /content/drive/My Drive/Research/data/EEG/chb01/chb01_26.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 595199  =      0.000 ...  2324.996 secs...


<ipython-input-11-330b93c843df>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_location, preload=True)


Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 sec)

1 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 1 events and 15360 original time points ...
0 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: logratio)
The file being converted is chb01_18.edf
Extracting EDF parameters from /content/drive/My Drive/Research/data/EEG/chb01/chb01_18.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Read

<ipython-input-11-330b93c843df>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_location, preload=True)


Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 sec)

1 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 1 events and 15360 original time points ...
0 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: logratio)
The file being converted is chb01_21.edf
Extracting EDF parameters from /content/drive/My Drive/Research/data/EEG/chb01/chb01_21.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Read

...

Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 sec)

1 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 1 events and 15360 original time points ...
0 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: logratio)
The file being converted is chb15_54.edf
Extracting EDF parameters from /content/drive/My Drive/Research/data/EEG/chb15/chb15_54.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Read

<ipython-input-11-330b93c843df>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_location, preload=True)
<ipython-input-11-330b93c843df>:12: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4, --5
  raw = mne.io.read_raw_edf(file_location, preload=True)


Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 sec)

1 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 1 events and 15360 original time points ...
0 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: logratio)
The file being converted is chb15_62.edf
Extracting EDF parameters from /content/drive/My Drive/Research/data/EEG/chb15/chb15_62.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Read

<ipython-input-11-330b93c843df>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_location, preload=True)
<ipython-input-11-330b93c843df>:12: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4, --5
  raw = mne.io.read_raw_edf(file_location, preload=True)


Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 sec)

1 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 1 events and 15360 original time points ...
0 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: logratio)


In [ ]:
# Creates the temp arrays we will be needing 
seizure_temp_train = []
seizure_temp_test = []

# Loops throw each epochs in the seizure_train_data 
for epochs in seizure_train_data: 
    for epoch in epochs: 
        # For each eopoch saves the single epochs into the seizure_temp_train
        seizure_temp_train.append(epoch)
seizure_np_train = np.stack(seizure_temp_train, axis=0) # Concats the seizure_temp_train
# to get one big np array which stores the seizure train data

# Does same as above but this time for seizure test data 
for epochs in seizure_test_data: 
    for epoch in epochs: 
        seizure_temp_test.append(epoch)
seizure_np_test = np.stack(seizure_temp_test, axis=0)

In [ ]:
# Making sure the shape is what we expect 
seizure_np_train.shape 
seizure_np_test.shape

(42, 3, 230, 230)

In [ ]:
# Writting the seizure data into csv files 
output_location = "/content/drive/My Drive/Research/" # Place output folder here 

# Writes the seizure train data to csv remeber we need to reshape the data as 
# csv files can only store 2D data so we need to flatten our data that is what 
# that is why we have .reshape(-1, 3 * 230 * 230)
np.savetxt(output_location + 'seizure_train_data.csv', seizure_np_train.reshape(-1, 3 * 230 * 230), delimiter=',')

# Does the same as above for seizure test data
np.savetxt(output_location + 'seizure_test_data.csv', seizure_np_test.reshape(-1, 3 * 230 * 230), delimiter=',')

## Feature extraction for non-seziure (so basically same as above but for non-seizures)

In [ ]:
# Temporary the fetaure extracted images  
non_seizure_train_data = []
non_seizure_test_data = [] 
# Patient 15 onwards will be used for testing the other patients will be ued for training 

error_files = [] # The names of the files the caused errors will be written here 

# Gets all the patient dirs in the EEG dataset 
dirs = [dir for dir in os.listdir(EEG_data_file_path) if os.path.isdir(os.path.join(EEG_data_file_path,dir))]
for directory in dirs: # Loops over all the directories
    for filename in os.listdir(EEG_data_file_path + directory): # Loops over each file in directory
        if (filename.endswith(".edf")): # Sees if file is a edf (i.e does it contain eeg data)
            if filename not in seizure_files: # Sees if file does not contain seizures
                if int(filename[3:5]) >= 15: # Checks if patient number is 15 or greater 
                    try: 
                        print("The file being converted is", f[0])
                        # Note: we are running are function throw the entire fiel this time not just between specific time segments 
                        power = get_images_from_file(EEG_data_file_path + directory + "/" + filename, common_channels, entire_file=True)
                        non_seizure_test_data.append(power) # Append value to seizure 
                    except Exception as e:
                        print(e)  
                        error_files.append(f[0]) # Append filename that causes error
                else: 
                    try: 
                        print("The file being converted is", f[0])
                        # Note: we are running are function throw the entire fiel this time not just between specific time segments 
                        power = get_images_from_file(EEG_data_file_path + directory + "/" + filename, common_channels, entire_file=True) 
                        non_seizure_train_data.append(power) # Append value to seizure 
                    except Exception as e:
                        print(e)  
                        error_files.append(f[0]) # Append filename that causes error
            break  # The break is here as we only want one non seizure file from each patient

The file being converted is chb24_21.edf
Extracting EDF parameters from /content/drive/My Drive/Research/data/EEG/chb01/chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


<ipython-input-4-330b93c843df>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_location, preload=True)


Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 sec)

60 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 60 events and 15360 original time points ...
0 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: logratio)
The file being converted is chb24_21.edf
Extracting EDF parameters from /content/drive/My Drive/Research/data/EEG/chb02/chb02_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Re

<ipython-input-4-330b93c843df>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_location, preload=True)


Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 sec)

60 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 60 events and 15360 original time points ...
0 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: logratio)
The file being converted is chb24_21.edf
Extracting EDF parameters from /content/drive/My Drive/Research/data/EEG/chb03/chb03_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Re

...

Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 sec)

42 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 42 events and 15360 original time points ...
0 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: logratio)
The file being converted is chb24_21.edf
Extracting EDF parameters from /content/drive/My Drive/Research/data/EEG/chb24/chb24_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Re

<ipython-input-4-330b93c843df>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_location, preload=True)


Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 sec)

60 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 60 events and 15360 original time points ...
0 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: logratio)
The file being converted is chb24_21.edf
Extracting EDF parameters from /content/drive/My Drive/Research/data/EEG/chb15/chb15_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Re

<ipython-input-4-330b93c843df>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'-'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_location, preload=True)
<ipython-input-4-330b93c843df>:12: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  raw = mne.io.read_raw_edf(file_location, preload=True)


Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 sec)

60 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 60 events and 15360 original time points ...
0 bad epochs dropped
Missing channels from ch_names required by include:
['T8-P8-0']


In [ ]:
# Creates the temp arrays we will be needing 
non_seizure_temp_train = []
non_seizure_temp_test = []

# Loops throw each epochs in the non_seizure_train_data 
for epochs in non_seizure_train_data: 
    for epoch in epochs: 
        # For each eopoch saves the single epochs into the seizure_temp_train
        non_seizure_temp_train.append(epoch)
non_seizure_np_train = np.stack(non_seizure_temp_train, axis=0) # Concats the non_seizure_temp_train
# to get one big np array which stores the seizure train data

# Does same as above but this time for non seizure test data 
for epochs in non_seizure_test_data: 
    for epoch in epochs: 
        non_seizure_temp_test.append(epoch)
non_seizure_np_test = np.stack(non_seizure_temp_test, axis=0)

In [ ]:
# Pritning the shape to make sure it looks correct 
print(non_seizure_np_train.shape)
print(non_seizure_np_test.shape) 

(1212, 3, 230, 230)
(402, 3, 230, 230)


In [ ]:
# Writting the non seizure data into csv files 
output_location = "/content/drive/My Drive/Research/" # Place output folder here 

# Writes the non seizure train data to csv remeber we need to reshape the data as 
# csv files can only store 2D data so we need to flatten our data that is what 
# that is why we have .reshape(-1, 3 * 230 * 230)
np.savetxt(output_location + 'non_seizure_train_data.csv', non_seizure_np_train.reshape(-1, 3 * 230 * 230), delimiter=',')

# Does the same as above for non seizure test data
np.savetxt(output_location + 'non_seizure_test_data.csv', non_seizure_np_test.reshape(-1, 3 * 230 * 230), delimiter=',')